# 1) Setup

## Imports

In [1]:
# ----------------------------------------
# Imports
# ----------------------------------------
import os

from rdflib import Namespace, SH
from pyshacl import validate

from tasty import constants as tc
from tasty import graphs as tg

## Inputs

In [2]:
# ----------------------------------------
# User Defined Variables
# ----------------------------------------

output_directory = os.path.join(os.path.abspath(''), '../tests/test_output')
tasty_main_directory = os.path.join(os.path.abspath(''), "../")
# print(tasty_main_directory)

SHAPE = 'NREL-VAV-Test-Shape'
SAMPLE = 'NREL-Equip-Test'
input_namespace_uri = 'urn:sample/'

data_graph_filename = 'tests/test_input/data_test.ttl'
shapes_graph_filename = 'tests/test_input/shapes_test.ttl'


# 2) Main Code

## Definitions

In [3]:
# ----------------------------------------
# Variables and Constants
# ----------------------------------------

NAMESPACE = Namespace(input_namespace_uri)

shape_name = tc.PH_SHAPES_NREL[SHAPE]
target_node = NAMESPACE[SAMPLE]

# ----------------------------------------
# Helper Function Definitions
# ----------------------------------------

def get_data_graph():
    n = tg.get_versioned_graph(tc.HAYSTACK, tc.V3_9_10)
    f = os.path.join(tasty_main_directory, data_graph_filename)
    n.parse(f, format='turtle')
    return n


def get_shapes_graph():
    g = tg.get_versioned_graph(tc.HAYSTACK, tc.V3_9_10)
    f = os.path.join(tasty_main_directory, shapes_graph_filename)
    g.parse(f, format='turtle')
    return g


def print_graph_to_file(g, filename):
    output_filename = os.path.join(output_directory, filename + ".ttl")
    g.serialize(output_filename, format='turtle')


def print_graph(g):
    print(g.serialize(format='turtle').decode('utf-8'))

## Setup Graphs

In [6]:
# ----------------------------------------
# Setup
# ----------------------------------------

# Data Graph
data_graph = get_data_graph()
print("...loaded data graph")

# Shapes Graph
shapes_graph = get_shapes_graph()
print("...loaded shapes graph")

# add Instance Equipment as target node to SHACL Equipment Shape
shapes_graph.add((shape_name, SH.targetNode, target_node))
print(f"\tadded '{target_node}' as target node to {shape_name}")
# add Instance Equipment as target node to SHACL Functional Groups Shapes
for s1, p1, o1 in shapes_graph.triples((shape_name, SH.node, None)):
    shapes_graph.add((o1, SH.targetNode, target_node))
    print(f"\tadded '{target_node}' as target node to {o1}")

# Ontology Graph
ont_graph = tg.load_ontology(tc.HAYSTACK, tc.V3_9_10)
print("...loaded ontology graph")

...loaded data graph
...loaded shapes graph
	added 'urn:sample/NREL-Equip-Test' as target node to https://project-haystack.org/datashapes/nrel#NREL-VAV-Test-Shape
	added 'urn:sample/NREL-Equip-Test' as target node to https://project-haystack.org/datashapes/nrel#TestFunctionalShape1
	added 'urn:sample/NREL-Equip-Test' as target node to https://project-haystack.org/datashapes/nrel#TestFunctionalShape2
...loaded ontology graph


# 3) Validation

## PySHACL Validation
Run the setup below

In [7]:
# ----------------------------------------
# Run pySCHACL Validation
# ----------------------------------------
result = validate(data_graph, shacl_graph=shapes_graph, ont_graph=ont_graph)
conforms, results_graph, results = result

print(f"   Conforms: {conforms}")



   Conforms: True
...printed results


## Print pySHACL Graphs and Results to File (Optional) 

In [5]:
# ----------------------------------------
# Print Output Files
# ----------------------------------------
# Print Results to File
fn = os.path.join(output_directory, "results.txt")
f = open(fn, "w")
f.write(results)
f.close()
print("...printed results")

# Print Graphs to File(s)
print_graph_to_file(data_graph, "data_graph")
print("...printed data graph")
print_graph_to_file(shapes_graph, "shapes_graph")
print("...printed shapes graph")
print_graph_to_file(results_graph, "results_graph")
print("...printed results graph")

## 3b) Brick Validation (Optional)

In [13]:
# ----------------------------------------
# Run BrickSchema Validation
# ----------------------------------------

from brickschema import Graph

# Set Up Graphs
dg = Graph()
df = os.path.join(tasty_main_directory, data_graph_filename)
dg.load_file(df)

sg = Graph()
sf = os.path.join(tasty_main_directory, shapes_graph_filename)
sg.load_file(sf)

valid, _, report = dg.validate(shape_graphs=[sg])
print(f"Brick Validation - Conforms: {valid}")
if not valid:
    print(report)

Brick Validation - Conforms: True
